In [1]:
import pandas as pd

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 8.2MB 10.0MB/s 


In [3]:
import pymorphy2
import math

In [93]:
def change(old, new):
  morph = pymorphy2.MorphAnalyzer()
  if (len(new.split(' ')) == 1):
    old_parse = morph.parse(old)[0]
    new_parse = morph.parse(new)[0]
    if (old_parse.tag.POS == 'NOUN' and new_parse.tag.POS == 'NOUN'):
      pad = old_parse.tag.case
      new = new_parse.inflect({pad}).word
      return new
    if (old_parse.tag.POS == 'INFN' and new_parse.tag.POS == 'INFN'):
      return new
    if (old_parse.tag.POS == 'INFN' and new_parse.tag.POS == 'VERB'):
      return new_parse.normal_form
    if (old_parse.tag.POS == 'VERB' and new_parse.tag.POS == 'INFN'):
      time = old_parse.tag.tense
      number = old_parse.tag.number
      if (number == 'plur'):
        new = new_parse.inflect({time, number}).word
        return new
      gender = old_parse.tag.gender
      new = new_parse.inflect({time, number, gender}).word
      return new
    if (old_parse.tag.POS == 'VERB' and new_parse.tag.POS == 'VERB'):
      time = old_parse.tag.tense
      number = old_parse.tag.number
      if (number == 'plur'):
        new = new_parse.inflect({time, number}).word
        return new
      person = old_parse.tag.person
      if (time == 'past'):
        gender = old_parse.tag.gender
        new = new_parse.inflect({time, number, gender, person}).word
        return new
      new = new_parse.inflect({time, number, person}).word
      return new
  else:
    old_parse = morph.parse(old)[0]
    if (old_parse.tag.POS == 'VERB'):
      k = 0
      for i in range (len(new.split(' '))):
        word = new.split(' ')[i]
        word_parse = morph.parse(word)[0]
        if (word_parse.tag.POS == 'VERB' or word_parse.tag.POS == 'INFN'):
          k = i
          break
      new_parse = morph.parse(new.split(' ')[k])[0]
      time = old_parse.tag.tense
      number = old_parse.tag.number
      if (number == 'plur'):
        new_change_word = new_parse.inflect({time, number}).word
        split_list = new.split(' ')
        split_list[k] = new_change_word
        new = ' '.join(split_list)
        return new
      if (time == 'past'):
        gender = old_parse.tag.gender
        new_change_word = new_parse.inflect({time, number, gender}).word
        split_list = new.split(' ')
        split_list[k] = new_change_word
        new = ' '.join(split_list)
        return new
      person = old_parse.tag.person
      new_change_word = new_parse.inflect({time, number, person}).word
      split_list = new.split(' ')
      split_list[k] = new_change_word
      new = ' '.join(split_list)
      return new
    if (old_parse.tag.POS == 'INFN'):
      k = 0
      for i in range (len(new.split(' '))):
        word = new.split(' ')[i]
        word_parse = morph.parse(word)[0]
        if (word_parse.tag.POS == 'INFN'):
          return new
        if (word_parse.tag.POS == 'VERB'):
          k = i
          break
      new_parse = morph.parse(new.split(' ')[k])[0]
      new_change_word = new_parse.normal_form
      split_list = new.split(' ')
      split_list[k] = new_change_word
      new = ' '.join(split_list)
      return new
    if (old_parse.tag.POS == 'NOUN'):
      k = 0
      for i in range (len(new.split(' '))):
        word = new.split(' ')[i]
        word_parse = morph.parse(word)[0]
        if (word_parse.tag.POS == 'NOUN'):
          k = i
          break
      pad = old_parse.tag.case
      new_parse = morph.parse(new.split(' ')[k])[0]
      new_change_word = new_parse.inflect({pad}).word
      if (k != 0):
        new_parse_adj = morph.parse(new.split(' ')[k-1])[0]
        new_change_word_adj = new_parse_adj.inflect({pad}).word
        split_list = new.split(' ')
        split_list[k-1] = new_change_word_adj
        split_list[k] = new_change_word
        new = ' '.join(split_list)
        return new
      split_list = new.split(' ')
      split_list[k] = new_change_word
      new = ' '.join(split_list)
      return new

In [124]:
class Tree():
    def __init__(self,root):
        self.root = root
        self.children = []
        self.Nodes = []

    def addNode(self,obj):
        self.children.append(obj)

    def changeAll(self):
      for child in self.children:
        child.changePattern()
      for child in self.children:
        if child.changeChildNodes() != None:
          child.getChildNodes()
        
    def getAllNodes(self):
        self.Nodes.append(self.root)
        for child in self.children:
            self.Nodes.append(child.data)
        for child in self.children:
            if child.getChildNodes(self.Nodes) != None:
                child.getChildNodes(self.Nodes)
        print(*self.Nodes, sep = "\n")
        print('Tree Size:' + str(len(self.Nodes)))

In [125]:
class Node():
    def __init__(self, data, template):
        self.data = data
        self.template = template
        self.arg = self.findArg(data)
        self.children = []

    def findArg(self, data):
      start = finish = 0
      for i in range(len(data)):
        if data[i] == '$' and start == 0:
          start = i + 1
          continue
        if data[i] == '$' and start != 0 and finish == 0:
          finish = i
          break
      return data[start:finish]

    def addNode(self,obj):
        self.children.append(obj)

    def changePattern(self):
      res = change(self.arg, self.template)
      start = finish = 0
      for i in range(len(self.data)):
        if self.data[i] == '$' and start == 0:
          start = i + 1
          continue
        if self.data[i] == '$' and start != 0 and finish == 0:
          finish = i
          break
      final = self.data[:start] + res + self.data[finish:]
      self.data = final
      #print(self.data)

    def getChildNodes(self,Tree):
        for child in self.children:
            if child.children:
                child.getChildNodes(Tree)
                Tree.append(child.data)
            else:
                Tree.append(child.data)

    def changeChildNodes(self):
        for child in self.children:
            if child.children:
                child.getChildNodes()
                child.changePattern()
            else:
                child.changePattern()

In [126]:
MyTree = Tree('Работаю утром')
MyTree.addNode(Node('Ем на $завтрак$ кашу', 'обед'))
MyTree.addNode(Node('Смотрю с утра $новости$', 'фильм'))
MyTree.addNode(Node('Подруга $ходила$ в магазин', 'поехать'))
MyTree.children[0].addNode(Node('Ем на $завтрак$ кашу', 'ужин'))
MyTree.getAllNodes()

Работаю утром
Ем на $завтрак$ кашу
Смотрю с утра $новости$
Подруга $ходила$ в магазин
Ем на $завтрак$ кашу
Tree Size:5


In [127]:
MyTree.changeAll()
MyTree.getAllNodes()

Работаю утром
Ем на $завтрак$ кашу
Смотрю с утра $новости$
Подруга $ходила$ в магазин
Ем на $завтрак$ кашу
Работаю утром
Ем на $обед$ кашу
Смотрю с утра $фильм$
Подруга $поехала$ в магазин
Ем на $ужин$ кашу
Tree Size:10
